In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
import copy
from sklearn.metrics import confusion_matrix
from features_general import aac_gen,dpc_gen,bin_aac
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import ExtraTreesClassifier 
import pickle
import warnings
warnings.filterwarnings('ignore')

/home/krkd/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
def hyperparams_call_save_models(X,Y,name,append):
#     clf_mlp=hyperparams_mlp(X,Y)
#     savemodel(append + 'mlp_' + name + '.pickle',clf_mlp)
    
    clf_svm=hyperparams_svm(X,Y)
    savemodel(append + 'svm_' + name + '.pickle',clf_svm)
    
#     clf_knn=hyperparams_knn(X,Y)
#     savemodel(append + 'knn_' + name + '.pickle',clf_knn)
    
#     clf_ridge=hyperparams_ridge_classifier(X,Y)
#     savemodel(append + 'ridge_' + name + '.pickle',clf_ridge)
    
#     clf_rf=hyperparams_random_forest(X,Y)
#     savemodel(append + 'rf_' + name + '.pickle',clf_rf)
    
#     clf_extra_trees=hyperparams_extra_trees(X,Y)
#     savemodel(append + 'extra_trees_' + name + '.pickle',clf_extra_trees)
    
    

In [4]:
def getX(X,indexes):
    ans=[]
    for i in indexes:
        ans+=[X[i]]
    return ans

In [5]:
def savemodel(strmodel,clf):
    # save the classifier
    with open(strmodel, 'wb') as fid:
        pickle.dump(clf, fid)    


In [6]:
def hyperparams_knn(X,Y):
    clf = KNeighborsClassifier()
    param_gridx = {  "n_neighbors":[ 1,3,5,7,9,11,13], "metric":["euclidean", "minkowski"] }  
    grid_search = GridSearchCV(clf, param_grid=param_gridx,cv=5)
    grid_search.fit(X,Y)
    print(grid_search.best_estimator_.get_params())
    return grid_search    

In [7]:
def hyperparams_svm(X,Y):
    clf = SVC(probability=True)
    #['rbf','linear','poly','sigmoid']
    #'kernel': ['rbf','poly']
    param_gridx = {  'gamma': [ 0.001,0.1,0.5,1,2], 'C': [0.5, 1,2,3,4]}  
    grid_search = GridSearchCV(clf, param_grid=param_gridx,cv=5,n_jobs=-1)
    grid_search.fit(X,Y)
    print(grid_search.best_estimator_.get_params())
    return grid_search   

In [8]:
def hyperparams_mlp(X,Y):
    clf = MLPClassifier()
    param_gridx = {'hidden_layer_sizes': [(10,20,30), (5,10,15), (10,10)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam','lbfgs']}
    grid_search = GridSearchCV(clf, param_grid=param_gridx,cv=5)
    grid_search.fit(X,Y)
    print(grid_search.best_estimator_.get_params())
    return grid_search   

In [9]:
def hyperparams_random_forest(X,Y):
    clf = RandomForestClassifier()
    depth=[]
    for i in range(1,20):
        if i%5==0:
            depth.append(i+1)
    
    param_gridx = { 'max_depth':depth,"max_features": depth,'n_estimators':[15,20,30],"min_samples_split":[2,3,10,20]} 
    grid_search = GridSearchCV(clf, param_grid=param_gridx,cv=5)
    grid_search.fit(X,Y)
    print(grid_search.best_estimator_.get_params())
    return grid_search

In [10]:
def hyperparams_ridge_classifier(X,Y):
    clf = RidgeClassifier()
    param_gridx = { 'alpha':[0.1,1.0,2.0,5.0,10.0,15.0,20.0,30.0,50.0],'tol':[0.0001,0.001,0.01,0.1,1]}  
    grid_search = GridSearchCV(clf, param_grid=param_gridx,cv=5)
    grid_search.fit(X,Y)
    print(grid_search.best_estimator_.get_params())
    return grid_search   
    

In [11]:
def hyperparams_extra_trees(X,Y):
    clf = ExtraTreesClassifier() 
    depth=[]
    for i in range(1,20):
        if i%5==0:
            depth.append(i+1)
    param_gridx = { 'max_depth':depth,"max_features": depth,'n_estimators':[15,20,30],"min_samples_split":[2,3,10,20]}  
    grid_search = GridSearchCV(clf, param_grid=param_gridx,cv=5)
    grid_search.fit(X,Y)
    print(grid_search.best_estimator_.get_params())
    return grid_search   
    

In [12]:
def roc_auc():
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], Y_scores[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        

In [13]:
def getVector(line,option1,option2,x=None,y=None):
    if option1=='aac':
        return aac_gen(line,option2,x,y)
    elif option1=='dpc':
        return dpc_gen(line,option2,x,y)
    elif option1=='bin':
        return bin_aac(line,option2,x,y)

In [14]:
def getXYforfeature(name,option1,option2,append,x=None,y=None):
    lennow=0
    if x!=None and y!=None: 
        lennow=max(x,y)
    elif x!=None:
        lennow=x
    elif y!=None:
        lennow=y
    
    positive_file=append+'/train/pos_train'
    negative_file=append+'/train/neg_train'
    X=[]
    Y=[]
    cnt=0
    f1=open(positive_file,'r')
    for line in f1:
        line=line.strip()
        line=line.upper()
        if line.count('U')==0 and len(line)>=lennow:
            X+=[getVector(line,option1,option2,x,y)]
            Y+=[+1]
    f1.close()
    
    f2=open(negative_file,'r')
    for line in f2:
        line=line.strip()
        line=line.upper()
        if line.count('U')==0 and len(line)>=lennow:
            X+=[getVector(line,option1,option2,x,y)]
            Y+=[-1]
    f2.close()  
    return X,Y

In [15]:
#ACP and Non ACP
root1='./ACPs and non-ACPS'
append_aac='./ACPs and non-ACPS/aac_models/'
append_dpc='./ACPs and non-ACPS/dpc_models/'

append_n5='./ACPs and non-ACPS/bin_n5_models/'
append_n10='./ACPs and non-ACPS/bin_n10_models/'
append_n15='./ACPs and non-ACPS/bin_n15_models/'

append_c5='./ACPs and non-ACPS/bin_c5_models/'
append_c10='./ACPs and non-ACPS/bin_c10_models/'
append_c15='./ACPs and non-ACPS/bin_c15_models/'

append_n5c5='./ACPs and non-ACPS/bin_nc5_models/'
append_n10c10='./ACPs and non-ACPS/bin_nc10_models/'
append_n15c15='./ACPs and non-ACPS/bin_nc15_models/'


In [16]:
#For AAC
X_aac,Y_aac=getXYforfeature('aac','aac','Normal',root1)

In [139]:
hyperparams_call_save_models(X_aac,Y_aac,'aac',append_aac)

{'bootstrap': True, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 13, 'max_features': 10, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 20, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 30, 'n_jobs': 1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [141]:
#For DPC 
X_dpc,Y_dpc=getXYforfeature('dpc','dpc','Normal',root1)

In [142]:
hyperparams_call_save_models(X_dpc,Y_dpc,'dpc',append_dpc)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
{'alpha': 5.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'to

In [20]:
#For N5
X_bin_n5,Y_bin_n5=getXYforfeature('bin','bin','N',root1,5)


In [21]:
hyperparams_call_save_models(X_bin_n5,Y_bin_n5,'bin_n5',append_n5)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
{'alpha': 2.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol

In [24]:
#For N10
X_bin_n10,Y_bin_n10=getXYforfeature('bin','bin','N',root1,10)


In [26]:
hyperparams_call_save_models(X_bin_n10,Y_bin_n10,'bin_n10',append_n10)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 0.5, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
{'alpha': 5.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 't

In [28]:
#For N15
X_bin_n15,Y_bin_n15=getXYforfeature('bin','bin','N',root1,15)

In [29]:
hyperparams_call_save_models(X_bin_n15,Y_bin_n15,'bin_n15',append_n15)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 0.5, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', '

In [22]:
#For C5
X_bin_c5,Y_bin_c5=getXYforfeature('bin','bin','C',root1,5)


In [23]:
hyperparams_call_save_models(X_bin_c5,Y_bin_c5,'bin_c5',append_c5)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (5, 10, 15), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 

In [26]:
#For C10
X_bin_c10,Y_bin_c10=getXYforfeature('bin','bin','C',root1,10)

In [27]:
hyperparams_call_save_models(X_bin_c10,Y_bin_c10,'bin_c10',append_c10)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (5, 10, 15), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 

In [32]:
#For C15
X_bin_c15,Y_bin_c15=getXYforfeature('bin','bin','C',root1,15)
len(X_bin_c15[0])

300

In [30]:
hyperparams_call_save_models(X_bin_c15,Y_bin_c15,'bin_c15',append_c15)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 20, 30), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 0.5, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto'

In [36]:
#For NC5
X_bin_nc5,Y_bin_nc5=getXYforfeature('bin','bin','NC',root1,x=5,y=5)

In [37]:
hyperparams_call_save_models(X_bin_nc5,Y_bin_nc5,'bin_nc5',append_n5c5)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
{'alpha': 15.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'to

In [40]:
#For NC10
X_bin_nc10,Y_bin_nc10=getXYforfeature('bin','bin','NC',root1,x=10,y=10)

In [41]:
hyperparams_call_save_models(X_bin_nc10,Y_bin_nc10,'bin_nc10',append_n10c10)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
{'alpha': 30.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol

In [45]:
#For NC15
X_bin_nc15,Y_bin_nc15=getXYforfeature('bin','bin','NC',root1,x=15,y=15)

In [46]:
hyperparams_call_save_models(X_bin_nc15,Y_bin_nc15,'bin_nc15',append_n15c15)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 20, 30), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 4, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto'

In [17]:
#ACP and Non ACP
root1='./ACPs and random peptides'
append_aac='./ACPs and random peptides/aac_models/'
append_dpc='./ACPs and random peptides/dpc_models/'

append_n5='./ACPs and random peptides/bin_n5_models/'
append_n10='./ACPs and random peptides/bin_n10_models/'
append_n15='./ACPs and random peptides/bin_n15_models/'

append_c5='./ACPs and random peptides/bin_c5_models/'
append_c10='./ACPs and random peptides/bin_c10_models/'
append_c15='./ACPs and random peptides/bin_c15_models/'

append_n5c5='./ACPs and random peptides/bin_nc5_models/'
append_n10c10='./ACPs and random peptides/bin_nc10_models/'
append_n15c15='./ACPs and random peptides/bin_nc15_models/'


In [51]:
#For AAC
X_aac,Y_aac=getXYforfeature('aac','aac','Normal',root1)

In [52]:
hyperparams_call_save_models(X_aac,Y_aac,'aac',append_aac)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 3, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 't

In [57]:
#For DPC 
X_dpc,Y_dpc=getXYforfeature('dpc','dpc','Normal',root1)

In [58]:
hyperparams_call_save_models(X_dpc,Y_dpc,'dpc',append_dpc)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 't

In [59]:
#For N5
X_bin_n5,Y_bin_n5=getXYforfeature('bin','bin','N',root1,5)

In [60]:
hyperparams_call_save_models(X_bin_n5,Y_bin_n5,'bin_n5',append_n5)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.5, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 15.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'to

In [61]:
#For N10
X_bin_n10,Y_bin_n10=getXYforfeature('bin','bin','N',root1,10)


In [62]:
hyperparams_call_save_models(X_bin_n10,Y_bin_n10,'bin_n10',append_n10)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 20.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'to

In [63]:
#For N15
X_bin_n15,Y_bin_n15=getXYforfeature('bin','bin','N',root1,15)

In [64]:
hyperparams_call_save_models(X_bin_n15,Y_bin_n15,'bin_n15',append_n15)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 20, 30), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 20.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto',

In [65]:
#For C5
X_bin_c5,Y_bin_c5=getXYforfeature('bin','bin','C',root1,5)


In [66]:
hyperparams_call_save_models(X_bin_c5,Y_bin_c5,'bin_c5',append_c5)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (5, 10, 15), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.5, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 20.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 

In [67]:
#For C10
X_bin_c10,Y_bin_c10=getXYforfeature('bin','bin','C',root1,10)

In [68]:
hyperparams_call_save_models(X_bin_c10,Y_bin_c10,'bin_c10',append_c10)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 10), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'lbfgs', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 't

In [69]:
#For C15
X_bin_c15,Y_bin_c15=getXYforfeature('bin','bin','C',root1,15)
len(X_bin_c15[0])

300

In [70]:
hyperparams_call_save_models(X_bin_c15,Y_bin_c15,'bin_c15',append_c15)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (5, 10, 15), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'lbfgs', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto',

In [71]:
#For NC5
X_bin_nc5,Y_bin_nc5=getXYforfeature('bin','bin','NC',root1,x=5,y=5)

In [73]:
hyperparams_call_save_models(X_bin_nc5,Y_bin_nc5,'bin_nc5',append_n5c5)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 20, 30), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 5.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', '

In [76]:
#For NC10
X_bin_nc10,Y_bin_nc10=getXYforfeature('bin','bin','NC',root1,x=10,y=10)
len(X_bin_nc10[0])

400

In [77]:
hyperparams_call_save_models(X_bin_nc10,Y_bin_nc10,'bin_nc10',append_n10c10)

{'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (5, 10, 15), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'sgd', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.1, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
{'alpha': 20.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', '

In [78]:
#For NC15
X_bin_nc15,Y_bin_nc15=getXYforfeature('bin','bin','NC',root1,x=15,y=15)

In [79]:
hyperparams_call_save_models(X_bin_nc15,Y_bin_nc15,'bin_nc15',append_n15c15)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (10, 20, 30), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 200, 'momentum': 0.9, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'lbfgs', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
{'C': 4, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'algorithm': 'auto', 'leaf_size': 30, 'metric': 'euclidean', 'metric_params': None, 'n_jobs': 1, 'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
{'alpha': 50.0, 'class_weight': None, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'aut

In [18]:
append_aac_common='./ACPs and non-ACPS/aac_models/'
append_dpc_common='./ACPs and non-ACPS/dpc_models/'





In [29]:
#For AAC N5
X_aac_n5,Y_aac_n5=getXYforfeature('aac','aac','N',root1,x=5)
hyperparams_call_save_models(X_aac_n5,Y_aac_n5,'aac_n5',append_aac_common)

#For AAC N10
X_aac_n10,Y_aac_n10=getXYforfeature('aac','aac','N',root1,x=10)
hyperparams_call_save_models(X_aac_n10,Y_aac_n10,'aac_n10',append_aac_common)

#For AAC N15
X_aac_n15,Y_aac_n15=getXYforfeature('aac','aac','N',root1,x=15)
hyperparams_call_save_models(X_aac_n15,Y_aac_n15,'aac_n15',append_aac_common)


#For AAC C5
X_aac_c5,Y_aac_c5=getXYforfeature('aac','aac','C',root1,x=5)
hyperparams_call_save_models(X_aac_c5,Y_aac_c5,'aac_c5',append_aac_common)

#For AAC C10
X_aac_c10,Y_aac_c10=getXYforfeature('aac','aac','C',root1,x=10)
hyperparams_call_save_models(X_aac_c10,Y_aac_c10,'aac_c10',append_aac_common)

#For AAC C15
X_aac_c15,Y_aac_c15=getXYforfeature('aac','aac','C',root1,x=15)
hyperparams_call_save_models(X_aac_c15,Y_aac_c15,'aac_c15',append_aac_common)


#For AAC NC5
X_aac_nc5,Y_aac_nc5=getXYforfeature('aac','aac','NC',root1,x=5,y=5)
hyperparams_call_save_models(X_aac_nc5,Y_aac_nc5,'aac_nc5',append_aac_common)

#For AAC NC10
X_aac_nc10,Y_aac_nc10=getXYforfeature('aac','aac','NC',root1,x=10,y=10)
hyperparams_call_save_models(X_aac_nc10,Y_aac_nc10,'aac_nc10',append_aac_common)

#For AAC NC15
X_aac_nc15,Y_aac_nc15=getXYforfeature('aac','aac','NC',root1,x=15,y=15)
hyperparams_call_save_models(X_aac_nc15,Y_aac_nc15,'aac_nc15',append_aac_common)




{'C': 0.5, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 4, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 

In [20]:

#For DPC N5
X_dpc_n5,Y_dpc_n5=getXYforfeature('dpc','dpc','N',root1,x=5)
hyperparams_call_save_models(X_dpc_n5,Y_dpc_n5,'dpc_n5',append_dpc_common)

#For DPC N10
X_dpc_n10,Y_dpc_n10=getXYforfeature('dpc','dpc','N',root1,x=10)
hyperparams_call_save_models(X_dpc_n10,Y_dpc_n10,'dpc_n10',append_dpc_common)

#For DPC N15
X_dpc_n15,Y_dpc_n15=getXYforfeature('dpc','dpc','N',root1,x=15)
hyperparams_call_save_models(X_dpc_n15,Y_dpc_n15,'dpc_n15',append_dpc_common)



#For DPC C5
X_dpc_c5,Y_dpc_c5=getXYforfeature('dpc','dpc','C',root1,x=5)
hyperparams_call_save_models(X_dpc_c5,Y_dpc_c5,'dpc_c5',append_dpc_common)

#For DPC C10
X_dpc_c10,Y_dpc_c10=getXYforfeature('dpc','dpc','C',root1,x=10)
hyperparams_call_save_models(X_dpc_c10,Y_dpc_c10,'dpc_c10',append_dpc_common)

#For DPC C15
X_dpc_c15,Y_dpc_c15=getXYforfeature('dpc','dpc','C',root1,x=15)
hyperparams_call_save_models(X_dpc_c15,Y_dpc_c15,'dpc_c15',append_dpc_common)



#For DPC NC5
X_dpc_nc5,Y_dpc_nc5=getXYforfeature('dpc','dpc','NC',root1,x=5,y=5)
hyperparams_call_save_models(X_dpc_nc5,Y_dpc_nc5,'dpc_nc5',append_dpc_common)

#For DPC NC10
X_dpc_nc10,Y_dpc_nc10=getXYforfeature('dpc','dpc','NC',root1,x=10,y=10)
hyperparams_call_save_models(X_dpc_nc10,Y_dpc_nc10,'dpc_nc10',append_dpc_common)

#For DPC NC15
X_dpc_nc15,Y_dpc_nc15=getXYforfeature('dpc','dpc','NC',root1,x=15,y=15)
hyperparams_call_save_models(X_dpc_nc15,Y_dpc_nc15,'dpc_nc15',append_dpc_common)




{'C': 4, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 0.5, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 

In [19]:

append_aac_common='./ACPs and random peptides/aac_models/'
append_dpc_common='./ACPs and random peptides/dpc_models/'








In [20]:
#For AAC N5
X_aac_n5,Y_aac_n5=getXYforfeature('aac','aac','N',root1,x=5)
hyperparams_call_save_models(X_aac_n5,Y_aac_n5,'aac_n5',append_aac_common)

#For AAC N10
X_aac_n10,Y_aac_n10=getXYforfeature('aac','aac','N',root1,x=10)
hyperparams_call_save_models(X_aac_n10,Y_aac_n10,'aac_n10',append_aac_common)

#For AAC N15
X_aac_n15,Y_aac_n15=getXYforfeature('aac','aac','N',root1,x=15)
hyperparams_call_save_models(X_aac_n15,Y_aac_n15,'aac_n15',append_aac_common)


#For AAC C5
X_aac_c5,Y_aac_c5=getXYforfeature('aac','aac','C',root1,x=5)
hyperparams_call_save_models(X_aac_c5,Y_aac_c5,'aac_c5',append_aac_common)

#For AAC C10
X_aac_c10,Y_aac_c10=getXYforfeature('aac','aac','C',root1,x=10)
hyperparams_call_save_models(X_aac_c10,Y_aac_c10,'aac_c10',append_aac_common)

#For AAC C15
X_aac_c15,Y_aac_c15=getXYforfeature('aac','aac','C',root1,x=15)
hyperparams_call_save_models(X_aac_c15,Y_aac_c15,'aac_c15',append_aac_common)


#For AAC NC5
X_aac_nc5,Y_aac_nc5=getXYforfeature('aac','aac','NC',root1,x=5,y=5)
hyperparams_call_save_models(X_aac_nc5,Y_aac_nc5,'aac_nc5',append_aac_common)

#For AAC NC10
X_aac_nc10,Y_aac_nc10=getXYforfeature('aac','aac','NC',root1,x=10,y=10)
hyperparams_call_save_models(X_aac_nc10,Y_aac_nc10,'aac_nc10',append_aac_common)

#For AAC NC15
X_aac_nc15,Y_aac_nc15=getXYforfeature('aac','aac','NC',root1,x=15,y=15)
hyperparams_call_save_models(X_aac_nc15,Y_aac_nc15,'aac_nc15',append_aac_common)




{'C': 0.5, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 4, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 

In [22]:

# #For DPC N5
# X_dpc_n5,Y_dpc_n5=getXYforfeature('dpc','dpc','N',root1,x=5)
# hyperparams_call_save_models(X_dpc_n5,Y_dpc_n5,'dpc_n5',append_dpc_common)

# #For DPC N10
# X_dpc_n10,Y_dpc_n10=getXYforfeature('dpc','dpc','N',root1,x=10)
# hyperparams_call_save_models(X_dpc_n10,Y_dpc_n10,'dpc_n10',append_dpc_common)

# #For DPC N15
# X_dpc_n15,Y_dpc_n15=getXYforfeature('dpc','dpc','N',root1,x=15)
# hyperparams_call_save_models(X_dpc_n15,Y_dpc_n15,'dpc_n15',append_dpc_common)



# #For DPC C5
# X_dpc_c5,Y_dpc_c5=getXYforfeature('dpc','dpc','C',root1,x=5)
# hyperparams_call_save_models(X_dpc_c5,Y_dpc_c5,'dpc_c5',append_dpc_common)

# #For DPC C10
# X_dpc_c10,Y_dpc_c10=getXYforfeature('dpc','dpc','C',root1,x=10)
# hyperparams_call_save_models(X_dpc_c10,Y_dpc_c10,'dpc_c10',append_dpc_common)

# #For DPC C15
# X_dpc_c15,Y_dpc_c15=getXYforfeature('dpc','dpc','C',root1,x=15)
# hyperparams_call_save_models(X_dpc_c15,Y_dpc_c15,'dpc_c15',append_dpc_common)



#For DPC NC5
X_dpc_nc5,Y_dpc_nc5=getXYforfeature('dpc','dpc','NC',root1,x=5,y=5)
hyperparams_call_save_models(X_dpc_nc5,Y_dpc_nc5,'dpc_nc5',append_dpc_common)

#For DPC NC10
X_dpc_nc10,Y_dpc_nc10=getXYforfeature('dpc','dpc','NC',root1,x=10,y=10)
hyperparams_call_save_models(X_dpc_nc10,Y_dpc_nc10,'dpc_nc10',append_dpc_common)

#For DPC NC15
X_dpc_nc15,Y_dpc_nc15=getXYforfeature('dpc','dpc','NC',root1,x=15,y=15)
hyperparams_call_save_models(X_dpc_nc15,Y_dpc_nc15,'dpc_nc15',append_dpc_common)




{'C': 1, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 4, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
{'C': 2, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
